# Effectiveness - Add multiple points

In [ ]:
import sys
sys.path.append("..")

from sklearn import svm
import numpy as np

import dynashap

from examples.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize
)

In [ ]:
# Read data
x_train, y_train, x_test, y_test, \
columns_name = \
    preprocess_data('train_80p_2c.csv', 'test_80p_2c.csv')

x_train = x_train[:,:2]
x_test = x_test[:,:2]

add_point_idxs = [5, 13]

# pick two points from test for add
add_points_x = x_test[add_point_idxs, :2]
add_points_y = y_test[add_point_idxs]

x_train_added = np.append(x_train[:, :2], add_points_x, axis=0)
y_train_added = np.append(y_train, add_points_y)

x_test_added = np.delete(x_test[:, :2], add_point_idxs, axis=0)
y_test_added = np.delete(y_test, add_point_idxs)

x_test = x_test_added
y_test = y_test_added


model = svm.SVC(decision_function_shape='ovo')

mc_plus_sv_82 = dynashap.mc_shap(x_train_added, y_train_added, x_test_added, y_test_added, model,
                                 1000*len(y_train_added), 4)
save_npy('mc_plus_sv_82.npy', mc_plus_sv_82)

# init sv
model = svm.SVC(decision_function_shape='ovo')

mc_plus_sv_80 = dynashap.mc_shap(x_train, y_train, x_test_added, y_test_added, model,
                                 1000*len(y_train), 4)
save_npy('mc_plus_sv_80.npy', mc_plus_sv_80)

model = svm.SVC(decision_function_shape='ovo')
mc_sv_82 = dynashap.mc_shap(x_train_added, y_train_added, x_test_added, y_test_added, model,
                           100*len(y_train_added), 1)
save_npy('mc_sv_82.npy', mc_sv_82)

# delta
init_sv = load_npy('mc_plus_sv_80.npy')

delta_shap = dynashap.DeltaShap(x_train, y_train, x_test_added, y_test_added, model,
                                init_sv)
sv_81 = delta_shap.add_single_point(add_points_x[0], add_points_y[0],
                            100*(len(y_train)+1))
delta_shap = dynashap.DeltaShap(np.append(x_train, [add_points_x[0]], axis=0),
                                np.append(y_train, [add_points_y[0]]),
                                x_test_added, y_test_added, model,
                                sv_81)
delta_sv_82 = delta_shap.add_single_point(add_points_x[1], add_points_y[1],
                            100*(len(y_train)+2))

save_npy('delta_sv_82.npy', delta_sv_82)

# pivot
init_sv = load_npy('mc_plus_sv_80.npy')

pivot_shap = dynashap.PivotShap(x_train, y_train, x_test_added, y_test_added, model,
                              init_sv)

lsv_80 = pivot_shap.prepare(1000*len(y_train))
save_npy('lsv_80.npy', lsv_80)

sv_81 = pivot_shap.add_single_point(add_points_x[0], add_points_y[0],
                            100*(len(y_train)+1), flags={'flag_lsv': True})

pivot_shap.x_train = np.append(x_train, [add_points_x[0]], axis=0)
pivot_shap.y_train = np.append(y_train, add_points_y[0])

pivot_sv_82 = pivot_shap.add_single_point(add_points_x[1], add_points_y[1],
                                        100*(len(y_train)+2), flags={'flag_lsv': True})

save_npy('pivot_sv_82.npy', pivot_sv_82)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_sv = load_npy('mc_plus_sv_80.npy')

# KNN
heur_shap = dynashap.HeurShap(x_train, y_train,
                              x_test_added, y_test_added,
                              model, init_sv,
                              params={'method': 'knn'})
heur_shap.prepare(params={'n_neighbors': 3})
knn_sv_82 = heur_shap.add_multi_points(add_points_x, add_points_y)
save_npy('knn_sv_82.npy', knn_sv_82)

# for simi
model = svm.SVC(decision_function_shape='ovo')

idxs = [13, 78]
svs = list()

for idx in idxs:
    sv = dynashap.mc_shap(np.delete(x_train, idx, axis=0),
                          np.delete(y_train, idx),
                          x_test_added, y_test_added,
                          model, 1000*79, 4)
    save_npy('mc_plus_sv_79_del'+str(idx)+'.npy', sv)
    svs.append(sv)

svs = np.asarray(svs)

save_npy('knn_plus_svs.npy', svs)


# KNN+
heur_shap = dynashap.HeurShap(x_train, y_train,
                              x_test_added, y_test_added,
                              model, init_sv,
                              params={'method': 'knn+'})

heur_shap.prepare(flags={'exact': False, 'train': False},
                  params={'n_neighbors': 3,
                          'simi_type': 'ed',
                          'f_shap': 'n*n',
                          'rela': ['poly', 1],
                          'train_idxs': [13, 78],
                          'train_svs': load_npy('knn_plus_svs.npy')})

knn_plus_sv_82 = heur_shap.add_multi_points(add_points_x, add_points_y)
save_npy('knn_plus_sv_82.npy', knn_plus_sv_82)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_sv = load_npy('mc_plus_sv_80.npy')
base_shap = dynashap.BaseShap(x_train, y_train, x_test_added,
                              y_test_added, model, init_sv)

# average based
base_sv_82_avg = base_shap.add_multi_points(add_points_x,
                                            add_points_y,
                                            params={'method': 'avg'})

# leave one out based
base_sv_82_loo = base_shap.add_multi_points(add_points_x,
                                            add_points_y,
                                            params={'method': 'loo'})

save_npy('base_sv_82_avg.npy', base_sv_82_avg)

save_npy('base_sv_82_loo.npy', base_sv_82_loo)

# comparison
def comp(base_v, comp_sv, name):
    var = variance(base_v, normalize(base_v, comp_sv))
    print("variance of %s   \t : %.10f" % (name, var))

mc_plus_sv_82 = load_npy('mc_plus_sv_82.npy')

comp(mc_plus_sv_82, load_npy('delta_sv_82.npy'), 'delta')
comp(mc_plus_sv_82, load_npy('pivot_sv_82.npy'), 'pivot')
comp(mc_plus_sv_82, load_npy('knn_sv_82.npy'), 'knn')
comp(mc_plus_sv_82, load_npy('knn_plus_sv_82.npy'), 'knn+')
comp(mc_plus_sv_82, load_npy('mc_sv_82.npy'), 'mc')
comp(mc_plus_sv_82, load_npy('base_sv_82_loo.npy'), 'base loo')
comp(mc_plus_sv_82, load_npy('base_sv_82_avg.npy'), 'base avg')